In [1]:
import sys
import pickle
import yaml
import time
import numpy as np
import os
import os.path as osp
import pandas
import torch

from utils import Options, overlap_ratio
from models.mdnet import MDNet, set_optimizer, BCELoss, Precision
from models.extractor import SampleGenerator, RegionDataset
opts = yaml.safe_load(open('options.yaml','r'))

In [2]:
## load OTB database
def load_database(path='./datasets'):
    def gen_config(seq_name):
        seq_home = path
        img_dir = os.path.join(seq_home, seq_name, 'img')
        gt_path = os.path.join(seq_home, seq_name, 'groundtruth_rect.txt')

        img_list = os.listdir(img_dir)
        img_list.sort()
        img_list = [os.path.join(img_dir, x) for x in img_list]
        
        gt = []
        with open(gt_path) as f:
            for line in f.readlines():
                if '\t' in line:
                    gt.append(list(map(int,line.strip().split('\t'))))
                elif ',' in line:
                    gt.append(list(map(int,line.strip().split(','))))
                elif ' ' in line:
                    gt.append(list(map(int,line.strip().split(' '))))
        gt = np.array(gt)
        # with open(gt_path) as f:
        #     print(f.readlines())
        #     gt = np.loadtxt((x.replace('\t',',') for x in f), delimiter=',')
        return img_list, gt
    f = open(osp.join(path,'tb_100.txt'))
    seq_list = f.readlines()
    names = sorted([x.split('\t')[0].strip() for x in seq_list])
    data = {}
    for name in names:
        img_list, gt = gen_config(name)
        if len(img_list) != gt.shape[0]:
            continue
        seq = {}
        seq['images'] = img_list
        seq['gt'] = gt
        data[name] = seq
    return data

In [3]:
## train_mdnet main
def train_mdnet():
    # Init dataset
    data = load_database()
    K = len(data)
    dataset = [None] * K
    for k, seq in enumerate(data.values()):
        dataset[k] = RegionDataset(seq['images'], seq['gt'], opts)

    # Init model
    model = MDNet(opts['init_model_path'], K)
    if opts['use_gpu']:
        model = model.cuda()
    model.set_learnable_params(opts['ft_layers'])

    # Init criterion and optimizer
    criterion = BCELoss()
    evaluator = Precision()
    optimizer = set_optimizer(model, opts['lr'], opts['lr_mult'])

    # Main trainig loop
    for i in range(opts['n_cycles']):
        print('==== Start Cycle {:d}/{:d} ===='.format(i + 1, opts['n_cycles']))
        if i in opts.get('lr_decay', []):
            print('decay learning rate')
            for param_group in optimizer.param_groups:
                param_group['lr'] *= opts.get('gamma', 0.1)

        # Training
        model.train()
        prec = np.zeros(K)
        k_list = np.random.permutation(K)
        for j, k in enumerate(k_list):
            tic = time.time()
            # training
            pos_regions, neg_regions = dataset[k].next()
            if opts['use_gpu']:
                pos_regions = pos_regions.cuda()
                neg_regions = neg_regions.cuda()
            pos_score = model(pos_regions)
            neg_score = model(neg_regions)

            loss = criterion(pos_score, neg_score)
            model.zero_grad()
            loss.backward()
            if 'grad_clip' in opts:
                torch.nn.utils.clip_grad_norm_(model.parameters(), opts['grad_clip'])
            optimizer.step()

            prec[k] = evaluator(pos_score, neg_score)

            toc = time.time()-tic
            print('Iter {:2d} (Domain {:2d}), Loss {:.3f}, Precision {:.3f}, Time {:.3f}'
                    .format(j, k, loss.item(), prec[k], toc))

        print('Mean Precision: {:.3f}'.format(prec.mean()))
        print('Save model to {:s}'.format(opts['model_path']))
        if opts['use_gpu']:
            model = model.cpu()
        states = {'shared_layers': model.layers.state_dict()}
        torch.save(states, opts['model_path'])
        if opts['use_gpu']:
            model = model.cuda()


if __name__ == "__main__":
    train_mdnet()

==== Start Cycle 1/50 ====
Iter  0 (Domain 65), Loss 1.032, Precision 0.250, Time 0.280
Iter  1 (Domain 49), Loss 0.818, Precision 0.219, Time 0.123
Iter  2 (Domain 66), Loss 0.680, Precision 0.281, Time 0.092
Iter  3 (Domain 30), Loss 0.619, Precision 0.312, Time 0.100
Iter  4 (Domain 46), Loss 0.663, Precision 0.375, Time 0.130
Iter  5 (Domain 21), Loss 0.687, Precision 0.188, Time 0.091
Iter  6 (Domain  6), Loss 0.803, Precision 0.125, Time 0.120
Iter  7 (Domain 61), Loss 0.547, Precision 0.188, Time 0.105
Iter  8 (Domain 75), Loss 0.406, Precision 0.531, Time 0.099
Iter  9 (Domain 28), Loss 0.431, Precision 0.406, Time 0.081
Iter 10 (Domain  5), Loss 0.574, Precision 0.281, Time 0.110
Iter 11 (Domain 51), Loss 0.572, Precision 0.500, Time 0.088
Iter 12 (Domain 13), Loss 0.395, Precision 0.531, Time 0.085
Iter 13 (Domain  8), Loss 0.577, Precision 0.438, Time 0.113
Iter 14 (Domain 90), Loss 0.466, Precision 0.438, Time 0.094
Iter 15 (Domain 80), Loss 0.241, Precision 0.781, Time 0.0

Iter 44 (Domain 79), Loss 0.078, Precision 0.969, Time 0.075
Iter 45 (Domain 13), Loss 0.230, Precision 0.812, Time 0.087
Iter 46 (Domain 20), Loss 0.410, Precision 0.812, Time 0.077
Iter 47 (Domain 45), Loss 0.167, Precision 0.875, Time 0.083
Iter 48 (Domain 37), Loss 0.128, Precision 0.906, Time 0.079
Iter 49 (Domain 58), Loss 0.274, Precision 0.781, Time 0.095
Iter 50 (Domain  6), Loss 0.120, Precision 0.969, Time 0.107
Iter 51 (Domain  0), Loss 0.480, Precision 0.562, Time 0.093
Iter 52 (Domain 52), Loss 0.092, Precision 0.906, Time 0.076
Iter 53 (Domain 46), Loss 0.100, Precision 0.875, Time 0.105
Iter 54 (Domain  3), Loss 0.434, Precision 0.719, Time 0.100
Iter 55 (Domain  1), Loss 0.100, Precision 0.938, Time 0.110
Iter 56 (Domain  7), Loss 0.064, Precision 0.969, Time 0.117
Iter 57 (Domain 25), Loss 0.179, Precision 0.938, Time 0.085
Iter 58 (Domain 63), Loss 0.258, Precision 0.781, Time 0.099
Iter 59 (Domain 76), Loss 0.156, Precision 0.906, Time 0.098
Iter 60 (Domain 59), Los

Iter 88 (Domain 13), Loss 0.112, Precision 0.938, Time 0.091
Iter 89 (Domain 70), Loss 0.401, Precision 0.594, Time 0.089
Iter 90 (Domain 36), Loss 0.047, Precision 0.969, Time 0.086
Mean Precision: 0.874
Save model to models/mdnet_otb.pth
==== Start Cycle 4/50 ====
Iter  0 (Domain 75), Loss 0.152, Precision 0.812, Time 0.088
Iter  1 (Domain 85), Loss 0.183, Precision 0.844, Time 0.077
Iter  2 (Domain  3), Loss 0.189, Precision 0.844, Time 0.093
Iter  3 (Domain 56), Loss 0.277, Precision 0.781, Time 0.077
Iter  4 (Domain  7), Loss 0.110, Precision 1.000, Time 0.103
Iter  5 (Domain 44), Loss 0.086, Precision 0.938, Time 0.098
Iter  6 (Domain 47), Loss 0.077, Precision 0.938, Time 0.103
Iter  7 (Domain 36), Loss 0.041, Precision 0.969, Time 0.095
Iter  8 (Domain 23), Loss 0.081, Precision 0.938, Time 0.086
Iter  9 (Domain 19), Loss 0.079, Precision 0.938, Time 0.082
Iter 10 (Domain 42), Loss 0.056, Precision 0.938, Time 0.072
Iter 11 (Domain 80), Loss 0.119, Precision 0.906, Time 0.084
I

Iter 40 (Domain 40), Loss 0.106, Precision 0.938, Time 0.085
Iter 41 (Domain 14), Loss 0.304, Precision 0.781, Time 0.095
Iter 42 (Domain  3), Loss 0.123, Precision 0.938, Time 0.088
Iter 43 (Domain 84), Loss 0.023, Precision 1.000, Time 0.086
Iter 44 (Domain 15), Loss 0.053, Precision 1.000, Time 0.104
Iter 45 (Domain 31), Loss 0.127, Precision 0.938, Time 0.092
Iter 46 (Domain  1), Loss 0.101, Precision 0.906, Time 0.107
Iter 47 (Domain  6), Loss 0.038, Precision 0.969, Time 0.108
Iter 48 (Domain 76), Loss 0.102, Precision 0.906, Time 0.086
Iter 49 (Domain 72), Loss 0.200, Precision 0.906, Time 0.075
Iter 50 (Domain 34), Loss 0.269, Precision 0.812, Time 0.081
Iter 51 (Domain 86), Loss 0.275, Precision 0.688, Time 0.078
Iter 52 (Domain 18), Loss 0.025, Precision 1.000, Time 0.083
Iter 53 (Domain 13), Loss 0.124, Precision 0.906, Time 0.079
Iter 54 (Domain 66), Loss 0.138, Precision 0.844, Time 0.079
Iter 55 (Domain 27), Loss 0.317, Precision 0.812, Time 0.075
Iter 56 (Domain 47), Los

Iter 84 (Domain 53), Loss 0.041, Precision 1.000, Time 0.080
Iter 85 (Domain 43), Loss 0.030, Precision 0.969, Time 0.072
Iter 86 (Domain 38), Loss 0.150, Precision 0.875, Time 0.074
Iter 87 (Domain 63), Loss 0.046, Precision 0.969, Time 0.089
Iter 88 (Domain 74), Loss 0.334, Precision 0.875, Time 0.089
Iter 89 (Domain 73), Loss 0.343, Precision 0.844, Time 0.091
Iter 90 (Domain 36), Loss 0.027, Precision 1.000, Time 0.087
Mean Precision: 0.912
Save model to models/mdnet_otb.pth
==== Start Cycle 7/50 ====
Iter  0 (Domain 87), Loss 0.061, Precision 0.969, Time 0.077
Iter  1 (Domain 67), Loss 0.155, Precision 0.906, Time 0.121
Iter  2 (Domain 63), Loss 0.103, Precision 0.906, Time 0.087
Iter  3 (Domain 71), Loss 0.021, Precision 1.000, Time 0.073
Iter  4 (Domain 70), Loss 0.227, Precision 0.750, Time 0.091
Iter  5 (Domain 17), Loss 0.059, Precision 0.969, Time 0.074
Iter  6 (Domain 61), Loss 0.150, Precision 0.844, Time 0.091
Iter  7 (Domain  6), Loss 0.129, Precision 0.938, Time 0.105
I

Iter 36 (Domain 76), Loss 0.060, Precision 0.969, Time 0.094
Iter 37 (Domain 11), Loss 0.164, Precision 0.875, Time 0.098
Iter 38 (Domain 17), Loss 0.034, Precision 1.000, Time 0.076
Iter 39 (Domain 50), Loss 0.041, Precision 1.000, Time 0.106
Iter 40 (Domain 55), Loss 0.259, Precision 0.844, Time 0.078
Iter 41 (Domain 85), Loss 0.061, Precision 0.969, Time 0.081
Iter 42 (Domain 60), Loss 0.033, Precision 0.969, Time 0.079
Iter 43 (Domain 27), Loss 0.046, Precision 0.969, Time 0.075
Iter 44 (Domain 37), Loss 0.045, Precision 1.000, Time 0.081
Iter 45 (Domain 40), Loss 0.025, Precision 1.000, Time 0.093
Iter 46 (Domain  7), Loss 0.043, Precision 1.000, Time 0.102
Iter 47 (Domain 62), Loss 0.105, Precision 0.969, Time 0.080
Iter 48 (Domain 65), Loss 0.094, Precision 0.938, Time 0.083
Iter 49 (Domain 33), Loss 0.075, Precision 0.969, Time 0.073
Iter 50 (Domain 51), Loss 0.068, Precision 0.969, Time 0.077
Iter 51 (Domain 69), Loss 0.104, Precision 0.938, Time 0.097
Iter 52 (Domain 46), Los

KeyboardInterrupt: 